In [1]:
import pandas as pd
from scipy import sparse
import time

In [2]:
train = pd.read_csv('../data/train.data')

In [3]:
up = [(str(row.user_id), str(row.product_id)) for row in train.itertuples()]

In [4]:
df_X = train.drop(['user_id', 'eval_set', 'product_id', 'label'], 1)
y = train['label']

In [5]:
#def df2csr(df):
#    return sparse.csr_matrix(df.astype(float).values)
#X = df2csr(df_X)

In [6]:
sz = -1
df_X_train = df_X[0: sz]
#X_train = X[0: sz] 
y_train = y[0: sz]

#df_X_test = df_X[sz: 2 * sz]
#X_test = X[sz: 2 * sz]
#y_test = y[sz: 2 * sz]

In [7]:
import lightgbm as lgb

In [8]:
d_train = lgb.Dataset(df_X_train, y_train, categorical_feature=['aisle_id', 'department_id'])

In [9]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 98
start_time = time.time()
bst = lgb.train(params, d_train, ROUNDS)
print(time.time() - start_time)

107.620867968


In [10]:
for k, v in zip(bst.feature_importance(), bst.feature_name()):
    print k,v

47 Unnamed:_0
70 order_id
1931 order_number
59 order_dow
91 order_hour_of_day
619 days_since_prior_order
113 ('puadd_to_cart_order',_'pumean')
217 ('pudays_since_prior_order',_'pumean')
2041 ('puorder_number',_'pumean')
689 ('pureordered',_'pusum')
19 ('pureordered',_'pucount')
32 ('pureordered',_'puratio')
27 ('p_order_id',_'p_nunique')
89 ('p_add_to_cart_order',_'p_mean')
32 ('p_user_id',_'p_nunique')
125 ('p_days_since_prior_order',_'p_mean')
30 ('p_reordered',_'p_sum')
4 ('p_reordered',_'p_count')
425 ('p_reordered',_'p_ratio')
125 ('plevel_user_id',_'plevel_1')
35 ('plevel_user_id',_'plevel_2')
38 ('plevel_user_id',_'plevel_3')
20 ('plevel_user_id',_'plevel_4')
25 ('plevel_user_id',_'plevel_5')
28 ('plevel_user_id',_'plevel_6')
23 ('plevel_user_id',_'plevel_7')
22 ('plevel_user_id',_'plevel_8')
25 ('plevel_user_id',_'plevel_9')
79 ('plevel_user_id',_'plevel_10')
51 ('aisle_reordered',_'aisle_sum')
29 ('aisle_reordered',_'aisle_count')
134 ('aisle_reordered',_'aisle_ratio')
53 ('de

In [11]:
#lightgbm
bst_y_train_pred_prob = bst.predict(df_X_train)
#bst_y_test_pred_prob = bst.predict(df_X_test)
from sklearn.metrics import f1_score, roc_auc_score
#print('test_score', roc_auc_score(y_test, bst_y_test_pred_prob))
print('train_score', roc_auc_score(y_train, bst_y_train_pred_prob))

('train_score', 0.83273261300870771)


In [12]:
"""
from xgboost import XGBClassifier
xgb = XGBClassifier()
start_time = time.time()
xgb.fit(X_train, y_train)
print(time.time() - start_time)
xgb_y_train_pred_prob = xgb.predict_proba(X_train)[:, 1]
xgb_y_test_pred_prob = xgb.predict_proba(X_test)[:, 1]
#xgboost
print ('test_score', roc_auc_score(y_test, xgb_y_test_pred_prob))
print ('train_score', roc_auc_score(y_train, xgb_y_train_pred_prob))
"""

"\nfrom xgboost import XGBClassifier\nxgb = XGBClassifier()\nstart_time = time.time()\nxgb.fit(X_train, y_train)\nprint(time.time() - start_time)\nxgb_y_train_pred_prob = xgb.predict_proba(X_train)[:, 1]\nxgb_y_test_pred_prob = xgb.predict_proba(X_test)[:, 1]\n#xgboost\nprint ('test_score', roc_auc_score(y_test, xgb_y_test_pred_prob))\nprint ('train_score', roc_auc_score(y_train, xgb_y_train_pred_prob))\n"

In [13]:
#xgboost
#xgb_y = xgb.predict_proba(X)[:, 1]

#lightgbm
bst_y = bst.predict(df_X)

In [14]:
orders = pd.read_csv('../csv/orders.csv')
order_products_train = pd.read_csv('../csv/order_products__train.csv')

In [15]:
result = orders[orders.eval_set == 'train'].set_index('order_id').join(order_products_train.set_index('order_id'))

In [16]:
fresult = result[result.reordered == 1]

In [17]:
from collections import defaultdict
expect = defaultdict(set)
for row in fresult.itertuples():
    expect[str(row.user_id)].add(str(row.product_id))

In [18]:
def f1(prediction, expect, verbose=False):
    prediction = [int(p) for p in prediction]
    expect = set([int(e) for e in expect])
    
    if verbose:
        print(prediction)
        print(expect)
    tp = 0
    for c in prediction:
        if c in expect:
            tp += 1
    if tp != 0:
        precision = tp * 1.0 / len(prediction)
        recall = tp * 1.0 / len(expect)
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0
    return f1

In [19]:
def validate(threshold, y_prob):
    prediction = defaultdict(set)
    for i, (p, row) in enumerate(zip(y_prob, up)):
        if p > threshold:
            prediction[row[0]].add(row[1])
    s = 0
    for i, (user_id, product_ids) in enumerate(expect.iteritems()):
        s += f1(prediction.get(user_id, []), product_ids)
    print('%s %s' % (threshold, s / len(expect)))

In [22]:
test = pd.read_csv('../data/test.data')

In [23]:
A = sparse.csr_matrix(test.drop(['user_id', 'eval_set', 'product_id'], 1).astype(float).values)

In [24]:
upt = [(str(row.user_id), str(row.product_id)) for row in test.itertuples()]

In [25]:
#xgboost
#b = xgb.predict_proba(A)[:,1]
#lightgbm
b = bst.predict(A)

In [26]:
uo = {str(int(float(u))): str(int(float(o))) for o, u in orders.reset_index().where(orders.eval_set == 'test')[['order_id', 'user_id']].astype(str).values if o != 'nan'}

In [33]:
threshold = 0.15
#for i in range(15, 20, 1):
#    validate(i * 0.01, bst_y) 

In [34]:
result = {o: set() for u,o in uo.iteritems() if o != 'nan'}
for i, (p, row) in enumerate(zip(b, upt)):
    if p > threshold:
        result[uo[row[0]]].add(row[1])

In [35]:
from __future__ import print_function
with open('../data/result_20170727_threshold_023_training_data.csv', 'w') as f:
    print('order_id,products', file=f)
    for k, v in result.iteritems():
        if len(v) == 0:
            print('%s,None' % k, file=f)
        else:
            print('%s,%s' % (k, ' '.join(v)), file=f)